# import

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import date
from dateutil.relativedelta import relativedelta

In [2]:
# API 키 불러오기

from dotenv import load_dotenv
import os

load_dotenv()
SERVICE_KEY = os.getenv("KOPIS_KEY")

if not SERVICE_KEY:
    raise ValueError("서비스 키가 설정되지 않았습니다. .env 파일을 확인하세요.")

# 공연 목록 조회

In [3]:
# xml 레코드 단위로 저장
def xml_to_records(xml_text, row_tag="db"):
    root = ET.fromstring(xml_text)
    recs = []
    for node in root.iter(row_tag):
       rec = {c.tag : (c.text or "").strip() for c in node}
       recs.append(rec)
    return recs 

In [4]:
# 월별 범위 지정 (api 최대 조회수가 100건인 점 고려)
def month_ranges(start="2014-01", end="2024-12"):
    y, m = map(int, start.split("-")); start_d = date(y, m, 1)
    y, m = map(int, end.split("-"));   end_d   = date(y, m, 1)
    cur = start_d
    while cur <= end_d:
        st = cur.strftime("%Y%m01")
        ed = (cur + relativedelta(months=1) - relativedelta(days=1)).strftime("%Y%m%d")
        yield st, ed, cur.year, cur.month
        cur += relativedelta(months=1)

In [5]:
# api 호출
def fetch_perf_list(stdate, eddate, shcate, signgucode, rows=100):
    """월 단위 공연목록 전부 가져오기"""
    url = "http://www.kopis.or.kr/openApi/restful/pblprfr"
    all_rows, cpage = [], 1
    while True:
        params = {
            "service": SERVICE_KEY,
            "stdate": stdate,
            "eddate": eddate,
            "cpage": cpage,
            "rows": rows,
            "shcate": shcate,
            "signgucode": signgucode
        }
        r = requests.get(url, params=params, timeout=30)
        r.raise_for_status()
        recs = xml_to_records(r.text, "db")
        if not recs: break
        all_rows.extend(recs)
        if len(recs) < rows: break
        cpage += 1
    return all_rows

In [6]:
# === 메인 루프 ===
genres = {"CCCD": "대중음악", "GGGA": "뮤지컬", "AAAA": "연극"}
regions = {"11": "서울", "28": "인천", "41": "경기도"}

all_data = []

In [20]:
for st, ed, year, month in month_ranges("2014-01", "2024-12"):
    for g in genres:
        for r in regions:
            try:
                recs = fetch_perf_list(st, ed, g, r)
                for rec in recs:
                    rec['year'], rec['month'] = year, month
                all_data.extend(recs)
                print(f"{year}-{month:02d} {genres[g]} {regions[r]}: {len(recs)}건")
            except Exception as e:
                print("Error:", year, month, g, r, e)

2014-01 대중음악 서울: 0건
2014-01 대중음악 인천: 0건
2014-01 대중음악 경기도: 0건
2014-01 뮤지컬 서울: 24건
2014-01 뮤지컬 인천: 0건
2014-01 뮤지컬 경기도: 0건
2014-01 연극 서울: 33건
2014-01 연극 인천: 0건
2014-01 연극 경기도: 0건
2014-02 대중음악 서울: 2건
2014-02 대중음악 인천: 0건
2014-02 대중음악 경기도: 0건
2014-02 뮤지컬 서울: 19건
2014-02 뮤지컬 인천: 0건
2014-02 뮤지컬 경기도: 0건
2014-02 연극 서울: 36건
2014-02 연극 인천: 0건
2014-02 연극 경기도: 0건
2014-03 대중음악 서울: 3건
2014-03 대중음악 인천: 0건
2014-03 대중음악 경기도: 0건
2014-03 뮤지컬 서울: 16건
2014-03 뮤지컬 인천: 0건
2014-03 뮤지컬 경기도: 0건
2014-03 연극 서울: 39건
2014-03 연극 인천: 0건
2014-03 연극 경기도: 0건
2014-04 대중음악 서울: 7건
2014-04 대중음악 인천: 0건
2014-04 대중음악 경기도: 0건
2014-04 뮤지컬 서울: 16건
2014-04 뮤지컬 인천: 0건
2014-04 뮤지컬 경기도: 0건
2014-04 연극 서울: 47건
2014-04 연극 인천: 0건
2014-04 연극 경기도: 0건
2014-05 대중음악 서울: 2건
2014-05 대중음악 인천: 0건
2014-05 대중음악 경기도: 0건
2014-05 뮤지컬 서울: 19건
2014-05 뮤지컬 인천: 0건
2014-05 뮤지컬 경기도: 0건
2014-05 연극 서울: 42건
2014-05 연극 인천: 0건
2014-05 연극 경기도: 0건
2014-06 대중음악 서울: 2건
2014-06 대중음악 인천: 0건
2014-06 대중음악 경기도: 0건
2014-06 뮤지컬 서울: 19건
2014-06 뮤지컬 인천: 0건
2014-06 뮤지컬 경기도: 0건


In [23]:
df = pd.DataFrame(all_data)

In [24]:
df.shape

(49992, 12)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49992 entries, 0 to 49991
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   mt20id     49992 non-null  object
 1   prfnm      49992 non-null  object
 2   prfpdfrom  49992 non-null  object
 3   prfpdto    49992 non-null  object
 4   fcltynm    49991 non-null  object
 5   poster     49992 non-null  object
 6   area       49991 non-null  object
 7   genrenm    49992 non-null  object
 8   openrun    49992 non-null  object
 9   prfstate   49992 non-null  object
 10  year       49992 non-null  int64 
 11  month      49992 non-null  int64 
dtypes: int64(2), object(10)
memory usage: 4.6+ MB


In [27]:
# id 중복 제거
df["mt20id"].nunique()

31814

In [28]:
# 공연 ID 기준 중복 제거
df_unique = df.drop_duplicates(subset='mt20id', keep="first").reset_index(drop=True)

In [29]:
print(df.shape) #중복 제거 전
print(df_unique.shape) #중복 제거 후

(49992, 12)
(31814, 12)


In [ ]:
# 연도별 딕셔너리로 저장
dfs_by_year = {year: data for year, data in df_unique.groupby("year")}

In [31]:
for y, d in dfs_by_year.items():
    print(y, d.shape)

2014 (288, 12)
2015 (1060, 12)
2016 (1160, 12)
2017 (1892, 12)
2018 (1531, 12)
2019 (3941, 12)
2020 (1862, 12)
2021 (3134, 12)
2022 (5060, 12)
2023 (5767, 12)
2024 (6119, 12)


# 공연 상세 조회

In [10]:
# 비동기로 공연 상세 api 조회

import aiohttp
import asyncio
from tqdm import tqdm

In [11]:
# xml 데이터 딕셔너리로 저장

def xml_to_dict(xml_text, row_tag="db"):
    root = ET.fromstring(xml_text)
    recs = []
    for node in root.iter(row_tag):
        rec = {c.tag : (c.text or "").strip() for c in node}
        recs.append(rec)
    return recs[0] if recs else {}

In [12]:
# mt20id 전달하여 상세조회 fetch + 많은 양 요청 처리를 위한 비동기
async def fetch_detail(session, mt20id, sem):
    url = f"http://www.kopis.or.kr/openApi/restful/pblprfr/{mt20id}"
    params={
        "service": SERVICE_KEY
    }
    async with sem:
        last_err = None
        for attempt in range(3): #최대 3번 재시도
            try:
                async with session.get(url, params=params, timeout=20) as resp:
                    text = await resp.text()
                    data = xml_to_dict(text)
                    data["mt20id"] = mt20id
                    return data
            except Exception as e:
                last_err = e
                if attempt < 2:
                    await asyncio.sleep(1)
                else: # 3번 다 실패하면 에러 기록 남김
                    return { "mt20id": mt20id, "error": str(last_err) }

In [13]:
# max_concurrency = 동시에 api 요청 보낼 최대 개수 (기본 5)
async def fetch_year_details(df_year, year, max_concurrency=5):
    sem = asyncio.Semaphore(max_concurrency) #최대 api 요청
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_detail(session, row["mt20id"], sem) for _, row in df_year.iterrows()]
        results = []
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc=f"Year {year}"):
            result = await f
            result["year"] = year
            results.append(result)
        return pd.DataFrame(results)

In [ ]:
# 2014만 실행 -- 요청 test 코드
df_2014 = dfs_by_year[2014]

In [44]:
df_detail_2014 = await fetch_year_details(df_2014, 2014, max_concurrency=5)

Year 2014:   0%|          | 0/288 [00:00<?, ?it/s]

Year 2014: 100%|██████████| 288/288 [00:29<00:00,  9.71it/s]


In [46]:
df_detail_2014.head()

,mt20id,prfnm,prfpdfrom,prfpdto,fcltynm,prfcast,prfcrew,prfruntime,prfage,entrpsnm,...,festival,musicallicense,musicalcreate,updatedate,prfstate,mt10id,dtguidance,styurls,relates,year
0,PF114212,호두까기 인형 [용산],2013.11.16,2014.01.05,국립중앙박물관 극장 용 (국립중앙박물관 극장 용),,,1시간 15분,24개월 이상,(주)피엠씨프러덕션(PMC Production),...,N,N,N,2019-07-25 10:03:14,공연완료,FC000008,"화요일 ~ 금요일(11:00,13:30), 토요일(11:00,13:30,16:00)...",,,2014
1,PF114868,100%광주,2014.04.26,2014.04.27,국립극장 (해오름극장),광주광역시 시민 100명,"헬가르트 하우크, 슈테판 카에기",1시간 40분,만 7세이상,(재)국립극장진흥재단,...,N,N,N,2019-07-25 10:03:14,공연완료,FC000003,"토요일(19:00), 일요일(15:00)",,,2014
2,PF122132,라이어 1탄 [해피씨어터],2014.11.29,2016.01.10,해피씨어터 (해피씨어터),"김종태, 배현일, 홍성인, 박주현, 조윤정, 김리나, 홍민희 등",,1시간 40분,만 12세 이상,파파프로덕션,...,N,N,N,2019-07-25 10:03:14,공연완료,FC001303,"월요일 ~ 목요일(17:00,20:00), 금요일(15:00,17:40,20:00)...",,,2014
3,PF212878,예술의전당 재즈페스타,2014.09.27,2014.09.28,예술의전당 [서울] (신세계스퀘어 야외무대),,,,,,...,N,N,N,2023-02-08 15:15:50,공연완료,FC000001,,,,2014
4,PF114932,"제10회 서울 아시테지 겨울축제, 붓바람",2014.01.06,2014.01.07,대학로예술극장 (대극장),,,1시간,36개월이상,극단 하땅세(하땅세 극장),...,Y,N,N,2020-09-03 17:35:02,공연완료,FC002633,"월요일 ~ 화요일(11:00,14:00)",,,2014


In [64]:
df_detail_2014.columns

Index(['mt20id', 'prfnm', 'prfpdfrom', 'prfpdto', 'fcltynm', 'prfcast',
       'prfcrew', 'prfruntime', 'prfage', 'entrpsnm', 'entrpsnmP', 'entrpsnmA',
       'entrpsnmH', 'entrpsnmS', 'pcseguidance', 'poster', 'sty', 'area',
       'genrenm', 'openrun', 'visit', 'child', 'daehakro', 'festival',
       'musicallicense', 'musicalcreate', 'updatedate', 'prfstate', 'mt10id',
       'dtguidance', 'styurls', 'relates', 'year'],
      dtype='object')

In [ ]:
# 연도별 상세조회 실행 함수
# api 요청량 초과 에러를 대비해 저장하면서
async def fetch_all_years(dfs_by_year, max_concurrency=5, save_dir="details_by_year"):
    os.makedirs(save_dir, exist_ok=True)
    all_results = {}
    
    for year, df_year in dfs_by_year.items():
        print(f"▶ Start fetching {year}, 공연 수: {len(df_year)}")
        
        df_detail = await fetch_year_details(df_year, year, max_concurrency=max_concurrency)
        all_results[year] = df_detail
        
        save_path = os.path.join(save_dir, f"detail_{year}.csv")
        df_detail.to_csv(save_path, index=False, encoding="utf-8-sig")
        print(f"✔ Finished {year}: {df_detail.shape}")
        
    return all_results

In [66]:
detail_dfs = await fetch_all_years(dfs_by_year, max_concurrency=5)

▶ Start fetching 2014, 공연 수: 288


Year 2014: 100%|██████████| 288/288 [00:28<00:00, 10.17it/s]


✔ Finished 2014: (288, 33)
▶ Start fetching 2015, 공연 수: 1060


Year 2015: 100%|██████████| 1060/1060 [01:41<00:00, 10.48it/s]


✔ Finished 2015: (1060, 33)
▶ Start fetching 2016, 공연 수: 1160


...

✔ Finished 2022: (5060, 34)
▶ Start fetching 2023, 공연 수: 5767


Year 2023: 100%|██████████| 5767/5767 [09:17<00:00, 10.34it/s] 


✔ Finished 2023: (5767, 33)
▶ Start fetching 2024, 공연 수: 6119


Year 2024: 100%|██████████| 6119/6119 [09:30<00:00, 10.73it/s]


✔ Finished 2024: (6119, 34)


# 공연별 통계 목록 조회

In [18]:
async def fetch_perf_stats(session, stdate, eddate, shcate, sem, rows=100):
    url = "https://kopis.or.kr/openApi/restful/prfstsPrfBy"
    all_rows, cpage = [], 1
    last_text = None
    
    async with sem:
        while True:
            params = {
                "service": SERVICE_KEY,
                "stdate": stdate,
                "eddate": eddate,
                "shcate": shcate,
                "rows": rows,
                "cpage": cpage,
            }
            
            async with session.get(url, params=params) as resp:
                text = await resp.text()
                
                print(f"[DEBUG] STATUS: {resp.status}, cpage={cpage}")
                # print("PARAMS:", params)
                # print("TEXT SAMPLE: ", text[:500])
                
                try:
                    if (text == last_text):
                        print(f"[STOP] 반복 응답 감지, cpage={cpage}")
                        break
                    
                    recs = xml_to_records(text, row_tag="prfst")
                    if not recs: break
                    all_rows.extend(recs)
                    if len(recs) < rows: break
                    cpage += 1
                except Exception as e:
                    print("XML Parse Error", e)
                    with open(f"error_{stdate}_{shcate}.xml", "w", encoding="utf-8") as f:
                        f.write(text)
                    break               
                  
    return all_rows

In [19]:
all_perf_stats = []

In [20]:
async with aiohttp.ClientSession() as session:
    sem = asyncio.Semaphore(3)  # 동시 요청 제한
    genres = {"CCCD": "대중음악", "GGGA": "뮤지컬", "AAAA": "연극"}
    for st, ed, year, month in month_ranges("2014-01", "2024-12"):
        for g in genres:
            try:
                recs = await fetch_perf_stats(session, st, ed, g, sem)
                for rec in recs:
                    rec['year'], rec['month'] = year, month
                    rec['shcate'] = g
                all_perf_stats.extend(recs)
                print(f"{year}-{month:02d} {genres[g]}: {len(recs)}건")
            except Exception as e:
                print("Error: ", year, month, g, e)

[DEBUG] STATUS: 200, cpage=1
2014-01 대중음악: 0건
[DEBUG] STATUS: 200, cpage=1
2014-01 뮤지컬: 11건
[DEBUG] STATUS: 200, cpage=1
2014-01 연극: 15건
[DEBUG] STATUS: 200, cpage=1
2014-02 대중음악: 2건
[DEBUG] STATUS: 200, cpage=1
2014-02 뮤지컬: 5건
[DEBUG] STATUS: 200, cpage=1
2014-02 연극: 14건
[DEBUG] STATUS: 200, cpage=1
2014-03 대중음악: 3건
[DEBUG] STATUS: 200, cpage=1
2014-03 뮤지컬: 2건
[DEBUG] STATUS: 200, cpage=1
2014-03 연극: 15건
[DEBUG] STATUS: 200, cpage=1
2014-04 대중음악: 7건
[DEBUG] STATUS: 200, cpage=1
2014-04 뮤지컬: 1건
[DEBUG] STATUS: 200, cpage=1
2014-04 연극: 20건
[DEBUG] STATUS: 200, cpage=1
2014-05 대중음악: 2건
[DEBUG] STATUS: 200, cpage=1
2014-05 뮤지컬: 3건
[DEBUG] STATUS: 200, cpage=1
2014-05 연극: 11건
[DEBUG] STATUS: 200, cpage=1
2014-06 대중음악: 2건
[DEBUG] STATUS: 200, cpage=1
2014-06 뮤지컬: 2건
[DEBUG] STATUS: 200, cpage=1
2014-06 연극: 8건
[DEBUG] STATUS: 200, cpage=1
2014-07 대중음악: 0건
[DEBUG] STATUS: 200, cpage=1
2014-07 뮤지컬: 6건
[DEBUG] STATUS: 200, cpage=1
2014-07 연극: 5건
[DEBUG] STATUS: 200, cpage=1
2014-08 대중음악: 1건
[DE

In [23]:
stats = pd.DataFrame(all_perf_stats)

In [25]:
stats.columns

Index(['mt20id', 'cate', 'prfdtcnt', 'prfnm', 'fcltynm', 'entrpsnm',
       'prfpdfrom', 'prfpdto', 'year', 'month', 'shcate'],
      dtype='object')

# 조인

In [31]:
# 연도별 detail_df 파일 합치기
import glob

detail_files = glob.glob("../details_by_year/detail_*.csv")

In [32]:
detail_files

['../details_by_year\\detail_2014.csv',
 '../details_by_year\\detail_2015.csv',
 '../details_by_year\\detail_2016.csv',
 '../details_by_year\\detail_2017.csv',
 '../details_by_year\\detail_2018.csv',
 '../details_by_year\\detail_2019.csv',
 '../details_by_year\\detail_2020.csv',
 '../details_by_year\\detail_2021.csv',
 '../details_by_year\\detail_2022.csv',
 '../details_by_year\\detail_2023.csv',
 '../details_by_year\\detail_2024.csv']

In [33]:
detail = pd.concat([pd.read_csv(f) for f in detail_files], ignore_index = True)

In [50]:
# mt20id 기준으로 병합
merged = pd.merge(
    detail,
    stats,
    on="mt20id",
    how="left",
    suffixes=("_detail", "_stats")
)

In [51]:
merged.columns

Index(['mt20id', 'prfnm_detail', 'prfpdfrom_detail', 'prfpdto_detail',
       'fcltynm_detail', 'prfcast', 'prfcrew', 'prfruntime', 'prfage',
       'entrpsnm_detail', 'entrpsnmP', 'entrpsnmA', 'entrpsnmH', 'entrpsnmS',
       'pcseguidance', 'poster', 'sty', 'area', 'genrenm', 'openrun', 'visit',
       'child', 'daehakro', 'festival', 'musicallicense', 'musicalcreate',
       'updatedate', 'prfstate', 'mt10id', 'dtguidance', 'styurls', 'relates',
       'year_detail', 'error', 'cate', 'prfdtcnt', 'prfnm_stats',
       'fcltynm_stats', 'entrpsnm_stats', 'prfpdfrom_stats', 'prfpdto_stats',
       'year_stats', 'month', 'shcate'],
      dtype='object')

In [52]:
rename_map = {
    "mt20id": "공연ID",
    "prfnm_detail": "공연명",
    "genrenm": "공연장르명",
    "prfstate": "공연상태",
    "prfpdfrom_detail": "공연시작일",
    "prfpdto_detail": "공연종료일",
    "fcltynm_detail": "공연시설명",
    "mt10id": "공연시설ID",
    "prfcast": "출연진",
    "prfcrew": "제작진",
    "prfruntime": "런타임",
    "prfage": "관람연령",
    "entrpsnmP": "제작사",
    "entrpsnmA": "기획사",
    "entrpsnmH": "주최",
    "entrpsnmS": "주관",
    "pcseguidance": "티켓가격",
    "poster": "포스터URL",
    "sty": "줄거리",
    "openrun": "오픈런",
    "area": "공연지역",
    "visit": "내한여부",
    "child": "아동공연여부",
    "daehakro": "대학로공연여부",
    "festival": "축제여부",
    "musicallicense": "뮤지컬라이센스",
    "musicalcreate": "뮤지컬창작",
    "updatedate": "최종수정일",
    "styurls": "소개이미지목록",
    "dtguidance": "공연시간",
    "cate": "장르(통계)",
    "prfnm_stats": "공연명(통계)",
    "prfdtcnt": "상연횟수",
    "prfpdfrom_stats": "공연시작일(통계)",
    "prfpdto_stats": "공연종료일(통계)",
    "fcltynm_stats": "공연시설명(통계)",
    "entrpsnm_stats": "제작/기획사(통계)",
    "year_stats": "연도(통계)",
    "month": "월(통계)",
    "shcate": "장르코드(통계)"
}

merged = merged.rename(columns=rename_map)

In [53]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35630 entries, 0 to 35629
Data columns (total 44 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   공연ID             35630 non-null  object 
 1   공연명              35628 non-null  object 
 2   공연시작일            35628 non-null  object 
 3   공연종료일            35628 non-null  object 
 4   공연시설명            35612 non-null  object 
 5   출연진              22837 non-null  object 
 6   제작진              15795 non-null  object 
 7   런타임              34072 non-null  object 
 8   관람연령             34912 non-null  object 
 9   entrpsnm_detail  15546 non-null  object 
 10  제작사              15546 non-null  object 
 11  기획사              4161 non-null   object 
 12  주최               21919 non-null  object 
 13  주관               15439 non-null  object 
 14  티켓가격             34555 non-null  object 
 15  포스터URL           34924 non-null  object 
 16  줄거리              2623 non-null   object 
 17  공연지역        

In [55]:
merged.drop(columns=['entrpsnm_detail', 'relates', 'year_detail', 'error' ], axis=1, inplace=True)

In [56]:
merged.columns

Index(['공연ID', '공연명', '공연시작일', '공연종료일', '공연시설명', '출연진', '제작진', '런타임', '관람연령',
       '제작사', '기획사', '주최', '주관', '티켓가격', '포스터URL', '줄거리', '공연지역', '공연장르명',
       '오픈런', '내한여부', '아동공연여부', '대학로공연여부', '축제여부', '뮤지컬라이센스', '뮤지컬창작', '최종수정일',
       '공연상태', '공연시설ID', '공연시간', '소개이미지목록', '장르(통계)', '상연횟수', '공연명(통계)',
       '공연시설명(통계)', '제작/기획사(통계)', '공연시작일(통계)', '공연종료일(통계)', '연도(통계)', '월(통계)',
       '장르코드(통계)'],
      dtype='object')

In [57]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35630 entries, 0 to 35629
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   공연ID        35630 non-null  object 
 1   공연명         35628 non-null  object 
 2   공연시작일       35628 non-null  object 
 3   공연종료일       35628 non-null  object 
 4   공연시설명       35612 non-null  object 
 5   출연진         22837 non-null  object 
 6   제작진         15795 non-null  object 
 7   런타임         34072 non-null  object 
 8   관람연령        34912 non-null  object 
 9   제작사         15546 non-null  object 
 10  기획사         4161 non-null   object 
 11  주최          21919 non-null  object 
 12  주관          15439 non-null  object 
 13  티켓가격        34555 non-null  object 
 14  포스터URL      34924 non-null  object 
 15  줄거리         2623 non-null   object 
 16  공연지역        35627 non-null  object 
 17  공연장르명       35628 non-null  object 
 18  오픈런         35628 non-null  object 
 19  내한여부        35628 non-nul

In [58]:
merged.to_csv("performance.csv", index=False, encoding="utf-8-sig")

# 공연시설정보

In [59]:
# 단일 시설 상세조회

async def fetch_facility_detail(session, mt10id, sem):
    url = f"http://www.kopis.or.kr/openApi/restful/prfplc/{mt10id}"
    params = {"service": SERVICE_KEY}
    async with sem:
        for _ in range(3):  # 최대 3회 재시도
            try:
                async with session.get(url, params=params, timeout=20) as resp:
                    text = await resp.text()
                    return {"mt10id": mt10id, **xml_to_dict(text)}
            except Exception as e:
                await asyncio.sleep(1)
        return {"mt10id": mt10id, "error": str(e)}

In [60]:
# 병렬 실행
async def fetch_all_facilities(mt10ids, max_concurrency=5):
    sem = asyncio.Semaphore(max_concurrency)
    async with aiohttp.ClientSession() as session:
        tasks = [fetch_facility_detail(session, fid, sem) for fid in mt10ids]
        results = []
        for f in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Facilities"):
            results.append(await f)
        return pd.DataFrame(results)

In [61]:
df = pd.read_csv("../datasets/KOPIS/performance_eda.csv")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_13764\1975926482.py:1: DtypeWarning: Columns (29,31,32,33,34,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../datasets/KOPIS/performance_eda.csv")


In [62]:
unique_facilities = df['공연시설ID'].dropna().unique()

In [63]:
unique_facilities

array(['FC001086', 'FC000003', 'FC002632', ..., 'FC004174', 'FC003865',
       'FC004171'], dtype=object)

In [64]:
# === 실행 ===
facility_df = await fetch_all_facilities(unique_facilities, max_concurrency=5)

print(f"총 {len(facility_df)}개 시설 정보 수집 완료")
facility_df.head()

Facilities: 100%|██████████| 1376/1376 [02:16<00:00, 10.09it/s]

총 1376개 시설 정보 수집 완료


,mt10id,fcltynm,mt13cnt,fcltychartr,opende,seatscale,telno,relateurl,adres,la,...,suyu,parkbarrier,restbarrier,runwbarrier,elevbarrier,parkinglot,mt13s,returncode,errmsg,responsetime
0,FC002047,이너프 라운지,1,기타(비공연장),2016,30,02-711-9245,http://enough.kr/,서울특별시 마포구 토정로17길 11 (신수동),37.545301900000000,...,N,N,N,N,N,Y,,NaN,NaN,NaN
1,FC002611,서울대학교,4,공공(기타),,115,02-880-5114,,서울특별시 관악구 관악로 1 (신림동),37.468038100000000,...,N,Y,N,N,N,Y,,NaN,NaN,NaN
2,FC003439,IVEX STUDIO,2,민간(대학로 외),2021,450,02-322-1112,https://ivexstudio.com/,경기도 광명시 양지로 17 (일직동),37.418680700000000,...,N,N,N,N,N,Y,,NaN,NaN,NaN
3,FC003573,아시아출판문화정보센터,1,민간(대학로 외),2003,0,031-955-0050,http://www.pajubookcity.org/Space/AsiaPublicat...,경기도 파주시 회동길 145 (문발동),37.708481200000000,...,N,N,N,N,N,N,,NaN,NaN,NaN
4,FC001177,광화문아트홀,1,공공(문예회관),2008,340,,,서울특별시 종로구 인왕산로1길 21 (사직동),37.574210200000000,...,N,Y,Y,Y,N,N,,NaN,NaN,NaN


In [67]:
facility_col_map = {
    "mt10id": "공연시설ID",
    "fcltynm": "공연시설명",
    "opende": "개관연도",
    "fcltychartr": "시설특성",
    "seatscale": "객석수",
    "mt13cnt": "공연장수",
    "telno": "전화번호",
    "relateurl": "홈페이지",
    "adres": "주소",
    "la": "위도",
    "lo": "경도",
    "restaurant": "레스토랑여부",
    "cafe": "카페여부",
    "store": "편의점여부",
    "nolibang": "놀이방여부",
    "suyu": "수유실여부",
    "parkbarrier": "장애시설_주차장",
    "restbarrier": "장애시설_화장실",
    "runwbarrier": "장애시설_경사로",
    "elevbarrier": "장애시설_엘리베이터",
    "parkinglot": "주차시설여부",
    "prfplcnm": "공연장명",
    "mt13id": "공연장ID",
    "stageorchat": "무대시설_오케스트라피트",
    "stagepracat": "무대시설_연습실",
    "stagedresat": "무대시설_분장실",
    "stageoutdrat": "무대시설_야외공연장",
    "disabledseatscale": "장애인석수",
    "stagearea": "무대넓이"
}


In [68]:
facility_df.rename(columns=facility_col_map, inplace=True)

In [ ]:
facility_df.to_csv("../datasets/KOPIS/facility_df.csv", index=False, encoding="utf-8-sig")

In [70]:
facility_df['주소'].value_counts().head(50)

주소
서울특별시 종로구 대학로11길 23 (명륜4가)       5
서울특별시 종로구 대학로8가길 30 (동숭동)        5
서울특별시 서대문구 연세로 50 (신촌동)          4
서울특별시 종로구 삼일대로 428 (낙원동)         4
서울특별시 종로구 대학로8가길 52 (동숭동)        3
서울특별시 광진구 능동로 216 (능동)           3
서울특별시 광진구 능동로 209 (군자동)          3
서울특별시 종로구 대학로 116 (동숭동)          3
서울특별시 종로구 동숭길 25 (동숭동)           3
서울특별시 서대문구 충정로 7 (충정로3가)         3
서울특별시 마포구 와우산로 76-1 (서교동)        3
서울특별시 종로구 동숭길 68 (동숭동)           2
서울특별시 종로구 동숭길 39 (동숭동)           2
서울특별시 은평구 통일로 684 (녹번동)          2
서울특별시 영등포구 영중로 15 (영등포동4가)       2
서울특별시 마포구 어울마당로 94-8 (서교동)       2
서울특별시 용산구 한강대로23길 55 (한강로3가)     2
서울특별시 종로구 인사동길 34-1 (관훈동)        2
서울특별시 종로구 대학로 144 (혜화동)          2
서울특별시 성동구 왕십리로 222 (사근동)         2
서울특별시 종로구 창경궁로35길 21 (혜화동)       2
서울특별시 마포구 양화로15안길 6 (서교동)        2
서울특별시 종로구 동숭길 74 (동숭동)           2
인천광역시 서구 서달로 190 (가정동)           2
서울특별시 서대문구 신촌로 129 (창천동)         2
경기도 성남시 분당구 수내로46번길 33 (수내동)     2
서울특별시 마포구 와우산로 117 (서교동)         2
서울특별시 영등포구 63로 50 (여의도동)         2
서울특별시 강북구 도봉로13가길

# 가격대별 통계목록

In [71]:
async def fetch_price_stats(session, stdate, eddate, shcate, sem):
    url = "http://www.kopis.or.kr/openApi/restful/prfstsPrice"
    params = {
        "service": SERVICE_KEY,
        "stdate": stdate,
        "eddate": eddate,
        "shcate": shcate
    }
    async with sem:
        try:
            async with session.get(url, params=params) as resp:
                text = await resp.text()
                return xml_to_records(text, row_tag="prfst")  # XML tag 맞춰야 함
        except Exception as e:
            print("Error:", stdate, eddate, shcate, e)
            return []


In [72]:
genres = {"CCCD": "대중음악", "GGGA": "뮤지컬", "AAAA": "연극"}
all_price_stats = []

async with aiohttp.ClientSession() as session:
    sem = asyncio.Semaphore(3)
    for st, ed, year, month in month_ranges("2014-01", "2024-12"):
        for g in genres:
            recs = await fetch_price_stats(session, st, ed, g, sem)
            for rec in recs:
                rec["year"], rec["month"], rec["shcate"] = year, month, g
            all_price_stats.extend(recs)
            print(f"{year}-{month:02d} {genres[g]}: {len(recs)}건")


2014-01 대중음악: 7건
2014-01 뮤지컬: 7건
2014-01 연극: 7건
2014-02 대중음악: 7건
2014-02 뮤지컬: 7건
2014-02 연극: 7건
2014-03 대중음악: 7건
2014-03 뮤지컬: 7건
2014-03 연극: 7건
2014-04 대중음악: 7건
2014-04 뮤지컬: 7건
2014-04 연극: 7건
2014-05 대중음악: 7건
2014-05 뮤지컬: 7건
2014-05 연극: 7건
2014-06 대중음악: 7건
2014-06 뮤지컬: 7건
2014-06 연극: 7건
2014-07 대중음악: 7건
2014-07 뮤지컬: 7건
2014-07 연극: 7건
2014-08 대중음악: 7건
2014-08 뮤지컬: 7건
2014-08 연극: 7건
2014-09 대중음악: 7건
2014-09 뮤지컬: 7건
2014-09 연극: 7건
2014-10 대중음악: 7건
2014-10 뮤지컬: 7건
2014-10 연극: 7건
2014-11 대중음악: 7건
2014-11 뮤지컬: 7건
2014-11 연극: 7건
2014-12 대중음악: 7건
2014-12 뮤지컬: 7건
2014-12 연극: 7건
2015-01 대중음악: 7건
2015-01 뮤지컬: 7건
2015-01 연극: 7건
2015-02 대중음악: 7건
2015-02 뮤지컬: 7건
2015-02 연극: 7건
2015-03 대중음악: 7건
2015-03 뮤지컬: 7건
2015-03 연극: 7건
2015-04 대중음악: 7건
2015-04 뮤지컬: 7건
2015-04 연극: 7건
2015-05 대중음악: 7건
2015-05 뮤지컬: 7건
2015-05 연극: 7건
2015-06 대중음악: 7건
2015-06 뮤지컬: 7건
2015-06 연극: 7건
2015-07 대중음악: 7건
2015-07 뮤지컬: 7건
2015-07 연극: 7건
2015-08 대중음악: 7건
2015-08 뮤지컬: 7건
2015-08 연극: 7건
2015-09 대중음악: 7건
2015-09 뮤지컬: 7건
2015-09

In [74]:
price_stats = pd.DataFrame(all_price_stats)

In [76]:
price_col_map = {
    "totnmrs": "총티켓판매수",      # 전체 티켓 판매 수량
    "cate": "장르",               # 장르명 (연극, 뮤지컬, 대중음악 등)
    "amount": "예매액",           # 예매된 총 금액
    "nmrs": "예매수",             # 예매 수량
    "nmrcancl": "취소수",         # 예매 취소 수량
    "amountsmratio": "예매액비중",  # 해당 가격대가 차지하는 예매액 비중(%)
    "price": "가격대"             # 가격대 구간 (예: 0원, 10,000원, 50,000원…)
}

In [77]:
price_stats.rename(columns=price_col_map, inplace=True)

In [79]:
price_stats.tail()

,장르,예매액,예매수,총티켓판매수,취소수,예매액비중,가격대,year,month,shcate
2767,연극,2947003,117788,78191,39597,26.3,3만원이상~5만원미만,2024,12,AAAA
2768,연극,1555824,47632,26562,21070,8.9,5만원이상~7만원미만,2024,12,AAAA
2769,연극,647949,13559,8416,5143,2.8,7만원이상~10만원미만,2024,12,AAAA
2770,연극,12704,139,105,34,0.0,10만원이상~15만원미만,2024,12,AAAA
2771,연극,7575,53,49,4,0.0,15만원이상,2024,12,AAAA


In [80]:
price_stats.to_csv("../datasets/KOPIS/price_stats.csv", index=False, encoding="utf-8-sig")

# 장르별 통계정보

In [82]:
async def fetch_cate_stats(session, stdate, eddate, sem):
    url = "http://www.kopis.or.kr/openApi/restful/prfstsCate"
    params = {
        "service": SERVICE_KEY,
        "stdate": stdate,
        "eddate": eddate,
    }
    async with sem:
        try:
            async with session.get(url, params=params) as resp:
                text = await resp.text()
                return xml_to_records(text, row_tag="prfst") 
        except Exception as e:
            print("Error:", stdate, eddate, e)
            return []

In [83]:
all_genre_stats = []

async with aiohttp.ClientSession() as session:
    sem = asyncio.Semaphore(3)
    for st, ed, year, month in month_ranges("2014-01", "2024-12"):
        for g in genres:
            recs = await fetch_price_stats(session, st, ed, g, sem)
            for rec in recs:
                rec["year"], rec["month"], rec["shcate"] = year, month, g
            all_genre_stats.extend(recs)
            print(f"{year}-{month:02d} {genres[g]}: {len(recs)}건")

2014-01 대중음악: 7건
2014-01 뮤지컬: 7건
2014-01 연극: 7건
2014-02 대중음악: 7건
2014-02 뮤지컬: 7건
2014-02 연극: 7건
2014-03 대중음악: 7건
2014-03 뮤지컬: 7건
2014-03 연극: 7건
2014-04 대중음악: 7건
2014-04 뮤지컬: 7건
2014-04 연극: 7건
2014-05 대중음악: 7건
2014-05 뮤지컬: 7건
2014-05 연극: 7건
2014-06 대중음악: 7건
2014-06 뮤지컬: 7건
2014-06 연극: 7건
2014-07 대중음악: 7건
2014-07 뮤지컬: 7건
2014-07 연극: 7건
2014-08 대중음악: 7건
2014-08 뮤지컬: 7건
2014-08 연극: 7건
2014-09 대중음악: 7건
2014-09 뮤지컬: 7건
2014-09 연극: 7건
2014-10 대중음악: 7건
2014-10 뮤지컬: 7건
2014-10 연극: 7건
2014-11 대중음악: 7건
2014-11 뮤지컬: 7건
2014-11 연극: 7건
2014-12 대중음악: 7건
2014-12 뮤지컬: 7건
2014-12 연극: 7건
2015-01 대중음악: 7건
2015-01 뮤지컬: 7건
2015-01 연극: 7건
2015-02 대중음악: 7건
2015-02 뮤지컬: 7건
2015-02 연극: 7건
2015-03 대중음악: 7건
2015-03 뮤지컬: 7건
2015-03 연극: 7건
2015-04 대중음악: 7건
2015-04 뮤지컬: 7건
2015-04 연극: 7건
2015-05 대중음악: 7건
2015-05 뮤지컬: 7건
2015-05 연극: 7건
2015-06 대중음악: 7건
2015-06 뮤지컬: 7건
2015-06 연극: 7건
2015-07 대중음악: 7건
2015-07 뮤지컬: 7건
2015-07 연극: 7건
2015-08 대중음악: 7건
2015-08 뮤지컬: 7건
2015-08 연극: 7건
2015-09 대중음악: 7건
2015-09 뮤지컬: 7건
2015-09

In [ ]:
genre_stats = pd.DataFrame(all_genre_stats)
